In [1]:
import os
import pandas as pd


In [ ]:
zillow_price_df = pd.read_csv("data/zillow_house_price.csv")
zillow_indicator_df = pd.read_csv("data/zillow_house_indicators.csv")
zillow_region_df = pd.read_csv("data/zillow_house_regions.csv")

In [12]:
# Merge the datasets
# First, merge ZILLOW DATA with ZILLOW INDICATORS on 'indicator_id'
merged_df = pd.merge(zillow_price_df, zillow_indicator_df, on='indicator_id', how='left')

# Then, merge the resulting DataFrame with ZILLOW REGIONS on 'region_id'
final_merged_df = pd.merge(merged_df, zillow_region_df, on='region_id', how='left')


In [25]:
zillow_state_filtered_df = final_merged_df[final_merged_df['region_type'] == "state"]


In [19]:
final_merged_df['category'].value_counts()

category
Home values            143998683
Inventory and sales      5539257
Rentals                   680021
Name: count, dtype: int64

In [23]:
final_merged_df['region_type'].value_counts()

region_type
city      52684479
zip       50024740
neigh     32575273
metro      8015666
county     6751002
state       166801
Name: count, dtype: int64

In [48]:
zillow_state_filtered_df.head()

,indicator_id,region_id,date,value,indicator,category,region_type,region
24,ZATT,32,2000-01-31,190316.235702,ZHVI All Homes- Top Tier Time Series ($),Home values,state,Missouri
25,ZATT,60,2000-01-31,207567.028078,ZHVI All Homes- Top Tier Time Series ($),Home values,state,Wisconsin
26,ZATT,10,2000-01-31,306954.843208,ZHVI All Homes- Top Tier Time Series ($),Home values,state,Colorado
27,ZATT,31,2000-01-31,224950.922544,ZHVI All Homes- Top Tier Time Series ($),Home values,state,Minnesota
28,ZATT,51,2000-01-31,214393.929836,ZHVI All Homes- Top Tier Time Series ($),Home values,state,South Carolina


In [49]:
zillow_state_filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 166801 entries, 24 to 83337687
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   indicator_id  166801 non-null  object 
 1   region_id     166801 non-null  int64  
 2   date          166801 non-null  object 
 3   value         166801 non-null  float64
 4   indicator     166801 non-null  object 
 5   category      166801 non-null  object 
 6   region_type   166801 non-null  object 
 7   region        166801 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 11.5+ MB


In [ ]:
zillow_state_filtered_df['region'].value_counts()

In [50]:
zillow_state_filtered_df['indicator'].value_counts()

indicator
ZHVI Single-Family Homes Time Series ($)             16709
ZHVI All Homes (SFR, Condo/Co-op) Time Series ($)    16709
ZHVI All Homes- Bottom Tier Time Series ($)          16709
ZHVI 3-Bedroom Time Series ($)                       16709
ZHVI 4-Bedroom Time Series ($)                       16709
ZHVI 5+ Bedroom Time Series ($)                      16706
ZHVI 2-Bedroom Time Series ($)                       16704
ZHVI All Homes- Top Tier Time Series ($)             16671
ZHVI Condo/Co-op Time Series ($)                     16626
ZHVI 1-Bedroom Time Series ($)                       16549
Name: count, dtype: int64

In [51]:
zillow_state_filtered_df['category'].value_counts()

category
Home values    166801
Name: count, dtype: int64

In [56]:
final_zillow_state_df = zillow_state_filtered_df[['indicator_id','region_id', 'date', 'value']]

In [57]:
final_zillow_state_df.head()

,indicator_id,region_id,date,value
24,ZATT,32,2000-01-31,190316.235702
25,ZATT,60,2000-01-31,207567.028078
26,ZATT,10,2000-01-31,306954.843208
27,ZATT,31,2000-01-31,224950.922544
28,ZATT,51,2000-01-31,214393.929836


In [59]:
final_zillow_state_df['date'].value_counts()

date
2013-05-31    510
2014-09-30    510
2015-03-31    510
2015-02-28    510
2015-01-31    510
             ... 
1996-06-30    465
1996-04-30    464
1996-03-31    464
1996-01-31    464
1996-02-29    464
Name: count, Length: 332, dtype: int64

In [2]:
#Download data from gdrive

import gdown

In [3]:
url = "https://drive.google.com/file/d/1MfTc_wxSmPrkQqb8a25pv6ciKiRYCND1/view?usp=sharing"

file_id = url.split("/")[-2]
file_id

'1MfTc_wxSmPrkQqb8a25pv6ciKiRYCND1'

In [4]:
prefix = 'https://drive.google.com/uc?/export=download&id='
gdown.download(prefix+file_id, "zillow_files.zip")

Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1MfTc_wxSmPrkQqb8a25pv6ciKiRYCND1
From (redirected): https://drive.google.com/uc?/export=download&id=1MfTc_wxSmPrkQqb8a25pv6ciKiRYCND1&confirm=t&uuid=67250db5-4468-4090-bfba-6a8096231721
To: /Users/hm/Documents/Github_Projects/Zillow-data-ML-project/notebook/zillow_files.zip
100%|██████████| 1.54G/1.54G [00:22<00:00, 68.8MB/s]


'zillow_files.zip'

In [5]:
%pwd

'/Users/hm/Documents/Github_Projects/Zillow-data-ML-project/notebook'

In [8]:
os.chdir("../")

In [9]:
%pwd

'/Users/hm/Documents/Github_Projects/Zillow-data-ML-project'

In [43]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [42]:
from ZillowHouseData.constants import *
from ZillowHouseData.utils.common import create_directories, read_yaml

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [44]:
import os, sys
import zipfile
import gdown
from ZillowHouseData.logger import logger
from ZillowHouseData.exception import CustomException

In [66]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise CustomException(e,sys)
        

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info("Extracted parent zip file")
        
        # Get the path to the current working directory
        os.chdir(unzip_path)
        current_working_directory = os.getcwd()

        # Iterate over the files in the current working directory
        for file in os.listdir(current_working_directory):
            # If the file is a zip file, unzip it
            if file.endswith(".zip"):
                with open(os.path.join(current_working_directory, file), 'rb') as f:
                    zipfile.ZipFile(f).extractall()
                    logger.info(f"{file} data files extracted successfully")

        # Deleting zipfiles            
        for file in os.listdir(current_working_directory):
            if file.endswith(".zip"):
                logger.info(f"Deleted {file} file")
                os.remove(file)
        
        # Reset cwd+
          
        os.chdir('../..')
        logger.info(f"working directory resetted to {os.getcwd()}")

In [67]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise CustomException(e,sys)

[2023-11-09 12:44:01,628: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-11-09 12:44:01,629: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-09 12:44:01,631: INFO: common: created directory at: artifacts]
[2023-11-09 12:44:01,632: INFO: common: created directory at: artifacts/data_ingestion]
[2023-11-09 12:44:01,632: INFO: 1720642734: Downloading data from https://drive.google.com/file/d/1MfTc_wxSmPrkQqb8a25pv6ciKiRYCND1/view?usp=sharing into file artifacts/data_ingestion/zillow_files.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1MfTc_wxSmPrkQqb8a25pv6ciKiRYCND1
From (redirected): https://drive.google.com/uc?/export=download&id=1MfTc_wxSmPrkQqb8a25pv6ciKiRYCND1&confirm=t&uuid=1a7b0dd2-558c-4dd6-928b-9ed6a4f177aa
To: /Users/hm/Documents/Github_Projects/Zillow-data-ML-project/artifacts/data_ingestion/zillow_files.zip
100%|██████████| 1.54G/1.54G [00:24<00:00, 61.8MB/s]

[2023-11-09 12:44:27,518: INFO: 1720642734: Downloaded data from https://drive.google.com/file/d/1MfTc_wxSmPrkQqb8a25pv6ciKiRYCND1/view?usp=sharing into file artifacts/data_ingestion/zillow_files.zip]


[2023-11-09 12:44:33,380: INFO: 1720642734: Extracted parent zip file]
[2023-11-09 12:44:33,382: INFO: 1720642734: zillow_indicators.zip data files extracted successfully]
[2023-11-09 12:44:33,417: INFO: 1720642734: zillow_regions.zip data files extracted successfully]
[2023-11-09 12:45:03,009: INFO: 1720642734: zillow_data.zip data files extracted successfully]
[2023-11-09 12:45:09,083: INFO: 1720642734: zillow_files.zip data files extracted successfully]
[2023-11-09 12:45:09,084: INFO: 1720642734: Deleted zillow_indicators.zip file]
[2023-11-09 12:45:09,085: INFO: 1720642734: Deleted zillow_regions.zip file]
[2023-11-09 12:45:09,085: INFO: 1720642734: Deleted zillow_data.zip file]
[2023-11-09 12:45:09,116: INFO: 1720642734: Deleted zillow_files.zip file]
[2023-11-09 12:45:09,205: INFO: 1720642734: working directory resetted to /Users/hm/Documents/Github_Projects/Zillow-data-ML-project]


In [62]:
import os, sys, json, shutil
import zipfile
import requests

class DataIngestion:

    def download_file(self):
        try:
            dataset_list = ["https://data.nasdaq.com/api/v3/datatables/ZILLOW/REGIONS?qopts.export=true&api_key=yaXdjWK7YAbeWZMoqphn", "https://data.nasdaq.com/api/v3/datatables/ZILLOW/INDICATORS?qopts.export=true&api_key=yaXdjWK7YAbeWZMoqphn"]
            for dataset_url in dataset_list:
                json_download_dir = "artifacts/data_ingestion/json"
                self.json_file_path = os.path.join(json_download_dir, "download_link_indic.json")
                self.zip_download_dir = "artifacts/data_ingestion/zip"
                self.zip_file_path = os.path.join(self.zip_download_dir, "data.zip")

                # Create directories if they don't exist
                if not os.path.exists(json_download_dir):
                    os.makedirs(json_download_dir)
                if not os.path.exists(self.zip_download_dir):
                    os.makedirs(self.zip_download_dir)

                # Download JSON file with download link
                response = requests.get(dataset_url)
                if response.status_code == 200:
                    with open(self.json_file_path, 'wb') as file:
                        file.write(response.content)

                # Read download link from JSON file
                with open(self.json_file_path, 'r') as json_file:
                    download_link = json.loads(json_file.read())["datatable_bulk_download"]["file"]["link"]

                # Download ZIP file
                response = requests.get(download_link)
                if response.status_code == 200:
                    with open(self.zip_file_path, 'wb') as file:
                        file.write(response.content)
                
                self.extract_zip_file()

        except Exception as e:
            raise e

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory
        Function returns None
        """
        try:

            unzip_path = "artifacts/data_ingestion"
            if not os.path.exists(unzip_path):
                os.makedirs(unzip_path)

            # unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)

            with zipfile.ZipFile('artifacts/data_ingestion/zip/data.zip', 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
                # logger.info("Extracted zip file")
            
            if os.path.isdir(os.path.dirname(self.json_file_path)):
                shutil.rmtree(os.path.dirname(self.json_file_path))

            if os.path.isdir(os.path.dirname(self.zip_file_path)):
                shutil.rmtree(os.path.dirname(self.zip_file_path))

        except Exception as e:
            raise CustomException(e,sys)

        # logger.info("Zip file extraction complete")

# Usage
data_ingestion = DataIngestion()
data_ingestion.download_file()
# data_ingestion.extract_zip_file() # Call this after confirming the download is successful


In [35]:
zip_download_dir = "artifacts/data_ingestion/zip"
os.path.join(zip_download_dir, "data.zip")

'artifacts/data_ingestion/zip/data.zip'

In [57]:
import shutil
json_download_dir = "artifacts/data_ingestion/json"
json_file_path = os.path.join(json_download_dir, "download_link_indic.json")

if os.path.isdir(os.path.dirname(json_file_path)):
    shutil.rmtree(os.path.dirname(json_file_path))

In [46]:
df = pd.read_csv("/Users/hm/Documents/Github_Projects/Zillow-data-ML-project-2/notebook/artifacts/data_ingestion/unzip/ZILLOW_DATA_962c837a6ccefddddf190101e0bafdaf.csv")

In [47]:
def calculate_min_max(df):
    """Calculates the min and max values for each column in the training data.

    Args:
        df (pandas.DataFrame): training data

    Returns:
        min_max_values (dict): min and max values for each column in the training data
    """
    min_max_values = df.agg(['min', 'max']).to_dict()
    return min_max_values

calculate_min_max(df)

{'indicator_id': {'min': 'CRAM', 'max': 'ZSFH'},
 'region_id': {'min': 3, 'max': 845172},
 'date': {'min': '1996-01-31', 'max': '2023-11-04'},
 'value': {'min': -864400.0, 'max': 40336513.0}}